# Person Re-identification using PySDK
This notebook demonstrates Person Re-Identification (Re-ID) using PySDK. Re-ID focuses on recognizing and matching people across different camera views based on their unique appearance, like clothing and body shape.

The basic pipeline works like this:
1. Detect people in the image using a person detection model.
2. Crop each detected person using the bounding box coordinates.
3. Apply the Person Re-ID model to the cropped images to extract the embeddings which can further be used to identify and match individuals across different images or camera views.

In [ ]:
import degirum as dg, degirum_tools

inference_host_address = "@local"
zoo_url = "degirum/hailo"
token = '' 
device_type = "HAILORT/HAILO8L"

# Person detection model name 
person_det_model_name = "yolov8n_relu6_person--640x640_quant_hailort_hailo8l_1"

# load AI model
person_det_model = dg.load_model(
    model_name=person_det_model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type
)

# Choose the Person reid model name 
person_reid_model_name = "osnet_x1_0_person_reid--256x128_quant_hailort_hailo8l_1"
# person_reid_model_name = "repvgg_a0_person_reid--256x128_quant_hailort_hailo8l_1" 

# load AI model
person_reid_model = dg.load_model(
    model_name=person_reid_model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type
)

#### Let's walk through a practical example of person re-identification.<br> 

## Example Scenario
We use two sets of images:
- **Set A:** Same person, different views.
- **Set B:** Another individual, different views.

The goal is to verify whether the ReID model can:

- Correctly match all images in Set A or B as the same person.
- Correctly distinguish Set B as a different person from Set A.

This simulates a real-world scenario where the model must recognize individuals across different camera views and lighting conditions.

In [ ]:
# Utility function for displaying images

import matplotlib.pyplot as plt

def display_images(images, titles="Images", figsize=(15, 5)):
    """
    Display a list of images in a single row using Matplotlib.
    
    Parameters:
    - images (list): List of images (NumPy arrays) to display.
    - titles (str or list): Either a single string for overall title, or list of titles for each image.
    - figsize (tuple): Size of the figure.
    """
    num_images = len(images)
    fig, axes = plt.subplots(1, num_images, figsize=figsize)
    if num_images == 1:
        axes = [axes]  # Make iterable for single image

    for i, (ax, image) in enumerate(zip(axes, images)):
        image_rgb = image[:, :, ::-1]  # Convert BGR to RGB
        ax.imshow(image_rgb)
        ax.axis('off')
        if isinstance(titles, list) and i < len(titles):
            ax.set_title(titles[i], fontsize=12)

    if isinstance(titles, str):
        fig.suptitle(titles, fontsize=16)

    plt.tight_layout()
    plt.show()

In [ ]:
#Image sources
image_source_1 = "../assets/person_1_multi_view.png"
image_source_2 = "../assets/person_2_multi_view.png"

# Detections
detections_1 = person_det_model(image_source_1)
detections_2 = person_det_model(image_source_2)

display_images([detections_1.image_overlay, detections_2.image_overlay], titles=["Person_1 Detections", "Person_2 Detections"], figsize=(10, 5))

In [ ]:
# Cropping

# Crops from the image_source_1
x1, y1, x2, y2 = map(int, detections_1.results[0]["bbox"])  # Convert bbox coordinates to integers
person1_crop1 = detections_1.image[y1:y2, x1:x2]  # Crop the person from the image

x1, y1, x2, y2 = map(int, detections_1.results[1]["bbox"])  # Convert bbox coordinates to integers
person1_crop2 = detections_1.image[y1:y2, x1:x2]  # Crop the person from the image

# Crops from the image_source_2
x1, y1, x2, y2 = map(int, detections_2.results[0]["bbox"])  # Convert bbox coordinates to integers
person2_crop1 = detections_2.image[y1:y2, x1:x2]  # Crop the person from the image

x1, y1, x2, y2 = map(int, detections_2.results[1]["bbox"])  # Convert bbox coordinates to integers
person2_crop2 = detections_2.image[y1:y2, x1:x2]  # Crop the person from the image

# Display person crops
display_images([person1_crop1, person1_crop2, person2_crop1, person2_crop2], titles=["Person1 Crop1","Person1 Crop2","Person2 Crop1", "Person2 Crop2"], figsize=(10, 5))

### Extracting embedding using a Re-Identification (ReID) model for each person crop

In [ ]:
import numpy as np
#Extract the embeddings for the image_source_1 that has two crops
embedding_person1_crop1 = np.asarray(person_reid_model(person1_crop1).results[0]["data"]).reshape(1, -1) # shape (1,512)
embedding_person1_crop2 = np.asarray(person_reid_model(person1_crop2).results[0]["data"]).reshape(1, -1) # shape (1,512)

#Extract the embeddings for the image_source_2 that detected two crops
embedding_person2_crop1 = np.asarray(person_reid_model(person2_crop1).results[0]["data"]).reshape(1, -1) # shape (1,512)
embedding_person2_crop2 = np.asarray(person_reid_model(person2_crop2).results[0]["data"]).reshape(1, -1) # shape (1,512)

### Calculating cosine similarity between the embeddings

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity between crops of the same person on different camera views
similarity_person1 = cosine_similarity(embedding_person1_crop1, embedding_person1_crop2)
print("Cosine similarity between Person1 crops (two images of the person1 with different camera views):",similarity_person1)

similarity_person2 = cosine_similarity(embedding_person2_crop1, embedding_person2_crop2)
print("Cosine similarity between Person2 crops (two images of the person2 with different camera views):", similarity_person2, "\n")


# Compute cosine similarity between crops of person1 and person2
similarity_p1c1_p2c1 = cosine_similarity(embedding_person1_crop1, embedding_person2_crop1)
print("Person 1 Crop 1 vs Person 2 Crop 1:", similarity_p1c1_p2c1)

similarity_p1c1_p2c2 = cosine_similarity(embedding_person1_crop1, embedding_person2_crop2)
print("Person 1 Crop 1 vs Person 2 Crop 2:", similarity_p1c1_p2c2)

similarity_p1c2_p2c1 = cosine_similarity(embedding_person1_crop2, embedding_person2_crop1)
print("Person 1 Crop 2 vs Person 2 Crop 1:", similarity_p1c2_p2c1)

similarity_p1c2_p2c2 = cosine_similarity(embedding_person1_crop2, embedding_person2_crop2)
print("Person 1 Crop 2 vs Person 2 Crop 2:", similarity_p1c2_p2c2)

### Interpreting Cosine Similarity Results
These results indicate that the Re-ID Osnet model effectively distinguishes between the same and different individuals:

- **Intra-person similarities** (Person 1 & Person 2 across views) are consistently **high (~0.87–0.88)**.
- **Inter-person similarities** (between different individuals) are **significantly lower (~0.44–0.55)**.

In person re-identification, it's common to apply a **similarity threshold** (typically in the range of ~0.7–0.8) to determine whether two embeddings represent the same individual. <br>
This threshold serves as a decision boundary: if the cosine similarity between two feature vectors (embeddings) exceeds the threshold, they are considered to belong to the same person; otherwise, they are treated as different individuals.

The ideal threshold may vary — it's typically fine-tuned on a validation set to achieve the best trade-off between false positives and false negatives, and it often depends on the specific model architecture and characteristics of the dataset. The choice involves balancing False positives and False negatives.